In [ ]:
import json

# Load GeoJSON file
with open("data/brownfield-land.geojson", 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Access features or properties
print(geojson_data['features'][:10])


In [ ]:
!pip install matplotlib

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Read GeoJSON file into GeoPandas DataFrame
file_path = r"data/brownfield-land.geojson"
geo_df = gpd.read_file(file_path)

# Plot the map
geo_df.plot(figsize=(10, 10), color='blue', edgecolor='black')
plt.title("Brownfield Land Map")
plt.show()


In [ ]:
import json

# Load GeoJSON file
with open("data/Lower_layer_Super_Output_Areas_December_2021_Boundaries_EW_BSC_V4_-4299016806856585929.geojson", 'r', encoding='utf-8') as f:
    geojson_lsoa = json.load(f)

# Access features or properties
print(geojson_lsoa['features'][:10])


In [ ]:
geojson_lsoa

In [ ]:
import geopandas as gpd
from shapely.geometry import shape

# Assuming geojson_lsoa is the loaded GeoJSON dictionary
features = geojson_lsoa['features']

# Extract geometries
geometries = [shape(feature['geometry']) for feature in features]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(features, geometry=geometries)

# Calculate centroids
gdf['centroid'] = gdf.geometry.centroid

# Print the centroids along with the ID
print(gdf[['properties', 'centroid']])


In [ ]:

gdf['properties'] = gdf['properties'].apply(lambda x: x['LSOA21CD'])

In [ ]:
gdf = gdf.rename(columns={'properties': 'LSOA code'})
gdf

In [ ]:
gdf.to_excel("data/gdf_export.xlsx", index=False)

In [ ]:
import pandas as pd

# Assuming geojson_lsoa is the loaded GeoJSON dictionary
features = geojson_lsoa['features']

# Extract geometries
geometries = [shape(feature['geometry']) for feature in features]

# Create a DataFrame
df = pd.DataFrame(features)

# Extract geometries and calculate centroids
df['geometry'] = geometries
df['centroid'] = df['geometry'].apply(lambda geom: geom.centroid)

# Print the centroids along with the ID
print(df[['properties', 'centroid']])


In [ ]:
df

In [ ]:
geo_df = gpd.read_file(file_path)

# Extract properties into a DataFrame
properties_df = pd.DataFrame(geo_df.drop(columns='geometry'))

# Calculate centroids
geo_df['centroid'] = geo_df.geometry.centroid

# Combine properties and centroids into a new DataFrame
combined_df = properties_df.copy()
combined_df['centroid'] = geo_df['centroid']

# Display the first few rows of the combined DataFrame
print(combined_df.head())

# Plot the map
geo_df.plot(figsize=(10, 10), color='blue', edgecolor='black')
plt.title("Brownfield Land Map")
plt.show()

In [ ]:
combined_df

In [ ]:
weather= pd.DataFrame(gdf['LSOA code'])

In [ ]:
weather

In [ ]:
import numpy as np

weather['avg_precip_mm'] = weather.apply(lambda x: np.random.uniform(100, 2274), axis=1)

In [ ]:
weather['cloud_okta'] = weather['avg_precip_mm'].apply(
    lambda x: np.random.randint(5, 9) if x > 1000 else 
              np.random.randint(2, 5) if 200 < x <= 1000 else 
              np.random.randint(0, 3)
)

In [ ]:
weather

In [ ]:
weather['wind_mph'] = weather['avg_precip_mm'].apply(
    lambda x: round(np.random.uniform(20, 63), 1) if x > 1800 else 
              round(np.random.uniform(0, 50), 1) if 800 < x <= 1800 else 
              round(np.random.uniform(0, 25), 1)
)

In [ ]:
weather

In [ ]:
weather.to_excel("data/dummy_weather.xlsx", index=False)

In [ ]:
bfl=pd.read_csv("data/brownfield-land.csv")

In [ ]:
bfl

In [ ]:
# dropping NAs from the dataset before converting to string, important has it will convert NAs to nan strings
bfl =bfl.dropna(subset=['notes'])

# counts for the history text column
bfl_t= bfl['notes'].str.split(expand=True).unstack().value_counts()

In [ ]:
!pip install plotly

In [ ]:
!pip install spacy

In [ ]:
!pip install nltk

In [ ]:
!pip install wordcloud

In [ ]:
# guidance for text preprocessing was used from Shanawad V. (2022) Elghali R. (2021), Anistropic (2018)
# nltk downloads
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)
#nlp = spacy.load('en_core_web_sm')


In [ ]:
# general libaries for basic data cleaning
import plotly.offline as py
import numpy as np
import pandas as pd
import os
import re
import string

from pprint import pprint



# nltyk packages and spacy. Helps with pre-processing text
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet, stopwords
from nltk.util import ngrams
from nltk import word_tokenize, pos_tag


#from nltk.tokenize import word_tokenize




# Data visualisation
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import pickle 

from pprint import pprint
from IPython.core.display import display, HTML
import seaborn as sns

In [ ]:
!pip install seaborn

In [ ]:
# nltk downloads
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)
#nlp = spacy.load('en_core_web_sm')
stopwords = set(stopwords.words('english'))

In [ ]:
from nltk.corpus import wordnet, stopwords
stopwords = set(stopwords.words('english'))

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords

In [ ]:
wnl = WordNetLemmatizer()
ps = PorterStemmer()
from nltk import word_tokenize, pos_tag

In [ ]:

# controlling how the words get converted
def tagger(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:

def drop_stopwords(text):
    # tokenises
    tokens = text
    dropped = [word for word in tokens if word.lower() not in stopwords]
    final_text = ' '.join(dropped)
    return final_text

# Shanawad V. (2022) Elghali R. (2021)
def text_clean(text):
    # convert text to string
    text = str(text)
    # identified some problematic names which could negatively impact. So combined

    text = re.sub(r'(Sector)\s+(\d+)', r'\1\2', text)

    # removing, non words, punction and web links and new lines

    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text) 
    text = re.sub(r'<.*?>+', ' ', text)
    text = re.sub(r'\n', ' ', text)
    # removing digits not attached to names
    text = re.sub(r'\b\d+\b', '', text)
    
    return text
def lemma_tag(sentence):
    sentence_words = word_tokenize(sentence)
    pos_tags = nltk.pos_tag(sentence_words)
    lemmatized_words = [wnl.lemmatize(word, pos=tagger(tag)) for word, tag in pos_tags]
    lemmatized_words =[word.lower() for word in lemmatized_words]
    # Join the lemmatized words into a single string
    lemmatized_sentence = ' '.join(lemmatized_words)
    return lemmatized_words

In [ ]:
word_tokenize("hello world")

In [ ]:
bfl.columns

In [ ]:
# dropping NAs from the dataset before converting to string, important has it will convert NAs to nan strings
bfl =bfl.dropna(subset=['notes'])

# counts for the history text column
bf11= bfl['notes'].str.split(expand=True).unstack().value_counts()


In [ ]:
# nltk downloads
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)
#nlp = spacy.load('en_core_web_sm')
#stopwords = set(stopwords.words('english'))

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
example=bfl['notes'][0]

print("Original:",example)
clean= text_clean(example)
print("\ncleaning:",clean)
lem= lemma_tag(clean)
print("\nLemmatisation:",lem)
stpword= drop_stopwords(lem)
print("\nStop word removal:",stpword)

In [ ]:
word_tokenize()

In [ ]:
bfl['lemma']=bfl['notes'].apply(text_clean).apply(lemma_tag).apply(drop_stopwords)

In [ ]:
text_lda_lemma = bfl['lemma'].values

In [ ]:
# word cloud
lda_text = ' '.join(bfl['lemma'])

# create word cload
wordcloud = WordCloud(width=1000, height=400).generate(lda_text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# word cloud
lda_text = ' '.join(bfl_t['notes'])

# create word cload
wordcloud = WordCloud(width=1000, height=400).generate(lda_text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()